In [1]:
import pandas as pd

In [83]:
df_raw = pd.read_excel('Data/Statement.xlsx',header=None)

In [84]:
df_raw

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1593,NaN,NaN,NaN,In case of questions about the statement you c...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1595,NaN,NaN,NaN,Statement generation date and time: 27.07.2025...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
new_columns = [
    'Date',
    'Online Date',
    'Amount',
    'Currency',
    'Credit',
    'Debit',
    'Applicable exchange rate',
    'Final Account Balance',
    'Transaction description'
]

In [86]:

# Step 2: Find the header row index where 'Date' appears
header_row_idx = None
for idx, row in df_raw.iterrows():
    if row.astype(str).str.contains('Date', case=False).any():
        header_row_idx = idx
        print(f"'Date' found in row: {header_row_idx}")
        break

if header_row_idx is None:
    raise ValueError("Header row with 'Date' not found.")
df = pd.read_excel('Data/Statement.xlsx', header=header_row_idx)
#df = df.dropna(subset=[df.columns[0]])
df = df.dropna(axis=1, how='all')


'Date' found in row: 32


In [87]:

# Step 3: Reload the Excel using the found header row
df = pd.read_excel('Data/Statement.xlsx', header=header_row_idx)

non_null_counts = df.notnull().sum()

# Calculate the mean of these counts
mean_non_nulls = non_null_counts.mean()
mean_non_nulls = mean_non_nulls/2
# Keep only columns where non-null count is >= mean
df = df.loc[:, non_null_counts >= mean_non_nulls]

# Change as per your real columns
df.columns = new_columns[:len(df.columns)]  # Avoid mismatch if fewer columns

# Step 5: (Optional) Drop any rows before the header row if still present (usually not needed now)
# If you want to be sure, you can reset index and drop rows with all NaNs or empty 'Date':
df = df.dropna(subset=['Date'])  # Remove rows where Date is NaN


In [88]:
df

,Date,Online Date,Amount,Currency,Credit,Debit,Applicable exchange rate,Final Account Balance,Transaction description
0,2025-06-02 00:00:00,NaN,411.0,AMD,411.0,NaN,NaN,NaN,NaN
7,2025-06-03 00:00:00,02.06.2025,150.0,AMD,NaN,150.0,NaN,NaN,NaN
14,2025-06-04 00:00:00,NaN,10000.0,AMD,10000.0,NaN,NaN,NaN,NaN
21,2025-06-05 00:00:00,NaN,2000.0,AMD,2000.0,NaN,NaN,NaN,NaN
28,2025-06-05 00:00:00,04.06.2025,1300.0,AMD,NaN,1300.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1519,2025-06-30 00:00:00,27.06.2025,7000.0,AMD,NaN,7000.0,NaN,NaN,NaN
1526,2025-06-30 00:00:00,27.06.2025,1300.0,AMD,NaN,1300.0,NaN,NaN,NaN
1533,2025-06-30 00:00:00,27.06.2025,1200.0,AMD,NaN,1200.0,NaN,NaN,NaN
1540,2025-06-30 00:00:00,27.06.2025,2500.0,AMD,NaN,2500.0,NaN,NaN,NaN


In [89]:
# Ensure Date column is datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Drop rows where Date is missing
df = df.dropna(subset=['Date'])

# Replace missing values in Credit and Debit with 0
df['Credit'] = pd.to_numeric(df['Credit'], errors='coerce').fillna(0)
df['Debit'] = pd.to_numeric(df['Debit'], errors='coerce').fillna(0)

# Group by Date and calculate sums
daily_summary = df.groupby(df['Date'].dt.date).agg({
    'Credit': 'sum',
    'Debit': 'sum'
}).reset_index().rename(columns={'Date': 'Day'})

print(daily_summary)


           Day   Credit     Debit
0   2025-06-02    411.0      0.00
1   2025-06-03      0.0    150.00
2   2025-06-04  10000.0      0.00
3   2025-06-05  17000.0   1600.00
4   2025-06-07  20000.0      0.00
5   2025-06-09   2000.0   3010.00
6   2025-06-10   5000.0  38380.00
7   2025-06-11      0.0   1570.00
8   2025-06-12   8000.0   8570.00
9   2025-06-13   8000.0   6260.00
10  2025-06-14  30000.0   5450.00
11  2025-06-15  10000.0      0.00
12  2025-06-16  30000.0  24013.00
13  2025-06-17   3000.0  38176.64
14  2025-06-18   8696.0  13715.00
15  2025-06-19   5000.0   5760.00
16  2025-06-20  11500.0   5660.00
17  2025-06-21  40000.0      0.00
18  2025-06-23  10000.0  37370.00
19  2025-06-24  10000.0   7150.00
20  2025-06-25  12000.0   6050.00
21  2025-06-26  14500.0  24060.00
22  2025-06-27  25002.0  26090.00
23  2025-06-28  10000.0      0.00
24  2025-06-30   2000.0  19410.00


C:\Users\gevor\AppData\Local\Temp\ipykernel_27476\1059535064.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Credit'] = pd.to_numeric(df['Credit'], errors='coerce').fillna(0)
C:\Users\gevor\AppData\Local\Temp\ipykernel_27476\1059535064.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Debit'] = pd.to_numeric(df['Debit'], errors='coerce').fillna(0)


In [91]:
daily_summary['Credit'].sum()

np.float64(292109.0)